In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import win32com.client
import os

params gniezno

In [2]:
VISUM_PATH = os.path.join(os.getcwd(),"data//visum.ver")
POI_CAT = 1
MTX_JRT = 20
MTX_PRT = 11

CZAS_PARKOWANIA = {"SPPN":6*60,
                   "POZA_SPPN":3*60}

params WBR

In [3]:
VISUM_PATH = "E://Niedzielski//WBR.ver"
POI_CAT = 101 #KINO_TEATR_MUZEUM
MTX_JRT = 10002002
MTX_PRT = 10002101

CZAS_PARKOWANIA = {"SPPN":6*60,
                   "POZA_SPPN":3*60}

In [4]:
Visum = win32com.client.Dispatch("Visum.Visum")  # uruchom Visum
Visum.LoadVersion(VISUM_PATH)  # zaladuj plik

In [5]:
POIs = pd.DataFrame(list(Visum.Net.POICategories.ItemByKey(POI_CAT).POIs.GetMultipleAttributes(["No", "ZoneID","Czas_Zone","SPPN"])),
                    columns = ["POI", "ZoneID","Czas_Zone","SPPN"])
POIs.ZoneID = POIs.ZoneID.astype(int)
POIs.POI = POIs.POI.astype(int)
POIs.SPPN = POIs.SPPN.astype(int)

In [6]:
POIs.head()

,POI,ZoneID,Czas_Zone,SPPN
0,30,1,12.011698,0
1,31,475,31.008541,0
2,32,1364,136.532092,0
3,33,286,94.610535,0
4,34,376,36.233270,0


In [7]:
cols = Visum.Net.Zones.GetMultiAttValues('No', False)
cols = [int(_[1]) for _ in cols]
mtx = Visum.Net.Matrices.ItemByKey(MTX_JRT)
vals = list(mtx.GetValuesFloat())
JRT = pd.DataFrame(vals, cols, cols)

In [8]:
mtx = Visum.Net.Matrices.ItemByKey(MTX_PRT)
vals = list(mtx.GetValuesFloat())
PRT = pd.DataFrame(vals, cols, cols)

In [9]:
del Visum

In [10]:
JRT = JRT.stack().reset_index()
JRT.columns = ['Z',"Do","Czas_PuT"]
PRT = PRT.stack().reset_index()
PRT.columns = ['Z',"Do","Czas_PrT"]

In [11]:
jk = pd.merge(JRT,POIs,how='outer',left_on="Do",right_on="ZoneID").dropna()
jk.POI = jk.POI.astype(int)
jk.SPPN = jk.SPPN.astype(int)

In [12]:
ki = pd.merge(POIs,JRT,how='outer',left_on="ZoneID",right_on="Z").dropna()
ki.POI = ki.POI.astype(int)
ki.SPPN = ki.SPPN.astype(int)

In [13]:
jki_JRT = pd.merge(jk,ki,on='POI')

In [14]:
del jki_JRT['SPPN_y']
del jki_JRT['Z_y']
del jki_JRT['ZoneID_x']
del jki_JRT['ZoneID_y']

In [15]:
jki_JRT.eval('Czas_PuT = Czas_PuT_x*60 + Czas_PuT_y*60 + 2*Czas_Zone_x', inplace = True)

In [17]:
del jki_JRT['Do_x']
del jki_JRT['Czas_PuT_x']
del jki_JRT['Czas_PuT_y']
del jki_JRT['Czas_Zone_x']
del jki_JRT['SPPN_x']
del jki_JRT['Czas_Zone_y']

In [18]:
jki_JRT.columns = ['Z_Rejon','POI','Do_Rejon','Czas_PuT']

In [30]:
jki_JRT.to_csv("data//jki_JRT.csv", chunksize = 100000)

KeyboardInterrupt: 

In [19]:
jk = pd.merge(PRT,POIs,how='outer',left_on="Do",right_on="ZoneID").dropna()
jk.POI = jk.POI.astype(int)
jk.SPPN = jk.SPPN.astype(int)

In [20]:
ki = pd.merge(POIs,PRT,how='outer',left_on="ZoneID",right_on="Z").dropna()
ki.POI = ki.POI.astype(int)
ki.SPPN = ki.SPPN.astype(int)

In [21]:
jki_PRT = pd.merge(jk,ki,on='POI')

In [33]:
del jki_PRT['SPPN_y']
del jki_PRT['Z_y']
del jki_PRT['ZoneID_x']
del jki_PRT['ZoneID_y']

KeyError: 'SPPN_y'

In [23]:
jki_PRT.eval('Czas_PrT = Czas_PrT_x*60 + Czas_PrT_y*60 + 2*Czas_Zone_x + 180 + 120*SPPN_x', inplace = True)

In [34]:
del jki_PRT['Do_x']
del jki_PRT['Czas_PrT_x']
del jki_PRT['Czas_PrT_y']
del jki_PRT['Czas_Zone_x']
del jki_PRT['SPPN_x']
del jki_PRT['Czas_Zone_y']

KeyError: 'Do_x'

In [35]:
jki_PRT.columns = ['Z_Rejon','POI','Do_Rejon','Czas_PrT']

In [ ]:
jki_PRT.to_csv("data//jki_PRT.csv")

In [36]:
jki = pd.merge(jki_PRT,jki_JRT,on=['Z_Rejon','POI','Do_Rejon'])

In [38]:
jki.to_csv("data//From_Via_To.csv", chunksize = 100000)